# AutoGen Demo

In [1]:
# To get stored prompts for later use
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Important Python modules

In [2]:
!pip install --quiet pyautogen litellm weaviate-client python-Levenshtein langchain langchain-openai langchain-community
!pip install --quiet protobuf==3.20.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.9/304.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.7/325.7 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

## Load the environment keys

In [3]:
from google.colab import userdata
import os
key = userdata.get("TENOPENAI")
os.environ["OPENAI_API_KEY"] = userdata.get("TENOPENAI")

## Import required modules

In [4]:
import json
import autogen
from langchain_community.vectorstores import Weaviate
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from litellm import embedding as test_embedding
import weaviate
from typing import List, Union, Dict, Annotated
import os

## setup

In [5]:
chat_model = "gpt-4o"
config_list = [{"model": chat_model, "api_key": os.environ["OPENAI_API_KEY"]}]
Tasks = ['SIMPLE QUESTION ANSWERING', 'REQUIREMENT GATHERING']
retriever_llm = "gpt-3.5-turbo-instruct"
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [58]:
import Levenshtein
import uuid
import re

def find_best_match(input_string, string_list):
    best_match = None
    best_score = float('inf')  # Lower score means better match

    for string in string_list:
        score = Levenshtein.distance(input_string, string)
        if score < best_score:
            best_score = score
            best_match = string

    return best_match

def read_file(file_name):
  file = open("/content/drive/MyDrive/Assistant10acad/prompts/" + file_name + ".txt", "r")
  content = file.read()
  file.close()
  return content


def create_vector_store(db_name):
  # vectorstore = Weaviate.from_documents(
  #     docs, embeddings, weaviate_url="https://test-kgmj8oyo.weaviate.network", index_name=db_name
  #     ,uuids=uuids
  # )

  client = weaviate.Client(url=userdata.get("ITALYLAWDB"))
  vectorstore = Weaviate(client, db_name , "text",embeddings,by_text=False)
  return vectorstore

def choose_retriever_types(retriever_type,vector_store):
  metadata_field_info = [
      AttributeInfo(
          name="codename",
          description="The 'Code' name of where the article lies. It is a section or part in a law.",
          type="string or list[string]",
      ),
      AttributeInfo(
          name="articlenumber",
          description="Article Number that the context is found at.",
          type="string",
      )
  ]
  document_content_description = "Articles of Italian law within each seaction of the law called 'Codes'"
  llm = OpenAI(model=retriever_llm,temperature=0)
  # if retriever_type == "self_query":
  #   retriever = SelfQueryRetriever.from_llm(
  #       llm, vector_store, document_content_description, metadata_field_info, verbose=True
  #   )
  if retriever_type == "vector_query_only":
    retriever = vector_store.as_retriever()
  # if retriever_type == "compressor":
  #    compressor_model = OpenAI(llm=retriever_llm,temperature=0);
  #    baseCompressor = LLMChainExtractor.fromLLM(compressor_model)
  #    retriever = ContextualCompressionRetriever(base_compressor=baseCompressor, base_retriever=vector_store.as_retriever());

  return retriever

def clean_json_string(json_string):
    pattern = r'^```json\s*(.*?)\s*```$'
    cleaned_string = re.sub(pattern, r'\1', json_string, flags=re.DOTALL)
    return cleaned_string.strip()

def divide_string(s):
    # Initialize a list to store the substrings
    substrings = []

    # Loop through the string, slicing 1500 characters at a time
    for i in range(0, len(s), 1500):
        substrings.append(s[i:i + 1500])

    return substrings

## worker and worker function

In [59]:
from collections import defaultdict
import datetime
timer_array = []

worker = autogen.AssistantAgent(
    name="worker",
    llm_config={"config_list": config_list, "cache_seed":42},
    code_execution_config=False,

    # the default system message of the AssistantAgent is overwritten here
    system_message="You are a helpful AI worker. You are a worker agent who does exactly what you are told to do.",
)

vectorstore = create_vector_store("Civil_Articles")
retriever = choose_retriever_types("vector_query_only",vectorstore)

def ask_worker(message , message_type, retrival):
    try:
      print(f"************** {message_type} *************************")
      print(message)
      print(retrival)
      # print(timer_array)
      timer_array.append([datetime.datetime.now(),assistant.get_total_usage()["gpt-4o-2024-05-13"]["prompt_tokens"],assistant.get_total_usage()["gpt-4o-2024-05-13"]["completion_tokens"],"assistant"])
      best_match = find_best_match(message_type, Tasks)

      prompt = read_file(best_match)

      context = ""
      if retrival == "TRUE":
        try:
          print("retrival started")
          context = "\n " + "Important laws to achieve the tasks: " + str(retriever.invoke(message))
          print("retrival ended")
        except:
          print("retrival failed")
          # context = ""

      if prompt != '':
        worker.update_system_message(prompt)
        worker.llm_config = {"config_list": config_list, "response_format":{ "type": "json_object" }}
        print("Prompt found for "+ message_type)
        # print(prompt + message + context)
      prepared_message = [{'content':  message + context ,  'role': 'assistant'}]
      print(" ************** Fully Prepared Worker Context *************************")
      print(prepared_message[0]["content"])
      print(" ************** Fully Prepared Worker Context *************************")
      # print(prepared_message
      history = defaultdict(list, {})
      worker._oai_messages = history
      result = worker.generate_reply(messages=prepared_message)
      # print(worker.chat_messages)

      timer_array.append([datetime.datetime.now(),worker.get_total_usage()["gpt-4o-2024-05-13"]["prompt_tokens"],worker.get_total_usage()["gpt-4o-2024-05-13"]["completion_tokens"],message_type])

      return result
    except Exception as e:
      print(e)
      return "Function Error"

# Creating our Assistant Agent and User Proxy

In [60]:
# - If the database retrieval does not provide important laws for the task, inform the user that you were not able to find the relevant law at that moment.
assistant = autogen.AssistantAgent(
    name="assistant",
    code_execution_config=False,
    system_message=
    """
    You are a Legal Assistant with advanced capabilities to solve complex problems. Your primary objective is to plan steps to appropriately solve users'\
     requests or tasks using worker function calling.

    The main problems you will be solving include:
    - Answering law questions

    Available Workers:
        - REQUIREMENT GATHERING - will enable you to ask relevant questions to the user based on the task
        - SIMPLE QUESTION ANSWERING - will enable you to answer simple questions and law retrievals to the user based on the task

    Your tasks:
    1. Outline the steps required to address the user's request.
    2. Use the `ask_worker` function to delegate each step, ensuring it returns the results upon task completion.
    3. VERY IMPORTANT: Before you return any result ensure that you do not need worker skills to perform a task.

    Key guidelines:
    - ALWAYS use the ask_worker function for different tasks and send it only the most important instructions and commands.
    - FORCE the ask_worker function to only give you the answer for your request without adding explanation or guidance what it achieved.
    - Give the worker the advantage of article retrival where it will get the latest laws regarding the task and be able to accomplish the task.
    - If results are based on an article or law codes, mention the articles in your response and the worker's response so that the user can verify the information.
    - DO NOT forget to put TERMINATE at the end, If you want feedback from user

    Communication:
    - Your answer should be constructed in the given language the user used to ask.
    - Before responding to the user make sure you used all the skills the worker function provides.
    - When communicating with the Worker make sure to send The most Important context to it.

    If you need to ask the user a question or you have finished the task add TERMINATE at the end.
    MUST - If the user send you an empty message just return your previous message and add TERMINATE at the end.
    """,

    llm_config={
        "temperature": 0,
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
        "functions": [
             {
                        "name": "ask_worker",
                        "description": "ask worker to:  1. generate questions for requirement gathering, 2. retrieve relevant laws based on the topic sent as message",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "message": {
                                    "type": "string",
                                    "description": "question to ask worker. Make sure the question include enough context, such as the required task. The worker does not know the conversation between you and the user, unless you share the conversation with the worker. Ensure the original question of the user is also passed as context such that the worker is generating intermediate steps necessary to address it. It is VERY IMPORTANT that you provide a clear and autoritative feedback to the worker to stay focused on answering the original question. If the worker is wandering off from the mission, terminate!"
                                },
                                "message_type": {
                                    "type": "string",
                                    "description": "Type of work you want it to achieve. Options are: 'REQUIREMENT GATHERING','SIMPLE QUESTION ANSWERING'"
                                },
                                "retrival":{
                                    "type": "string",
                                    "description": "Becomes 'TRUE' or 'FALSE' which would indicate to perform retrieval of relevant articles from the law to achieve the task"
                                },
                            },
                            "required": ["message","message_type","retrival"]
                        }
                        }
        ],
    },
)

def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()


# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    # code_execution_config={"work_dir": "planning"},
    function_map={"ask_worker": ask_worker },
)


## Setup Chat Environment

In [61]:
def chat_with_agent(message):
  # combined_message, lang = translate_to_dest(message,"en")
  user_proxy.initiate_chat(
      assistant,
      clear_history=False,
      silent=True,
      message=message
  )
  last_message = [messages for agent, messages in user_proxy.chat_messages.items()][0][-1]["content"].replace("TERMINATE" , "")
  # result_message, lang = translate_to_dest(last_message, lang)
  # print(result_message)
  print(last_message)

In [62]:
# !pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize the list to hold messages
messages = []


def handle_input(sender):
    global messages
    user_input = sender.value
    messages.append(f"User: {user_input}")
    chat_with_agent(user_input)
    messages.append("")
    display_messages()

# Function to display messages
def display_messages():

    text_input = widgets.Text(
        value='',
        placeholder='Type your message',
        description='Chat:',
        disabled=False
    )
    text_input.on_submit(handle_input)
    display(text_input)


display_messages()

Text(value='', description='Chat:', placeholder='Type your message')


>>>>>>>> EXECUTING FUNCTION ask_worker...
************** SIMPLE QUESTION ANSWERING *************************
What does the law say about civil rights and household rights?
TRUE
retrival started
retrival ended
 ************** Fully Prepared Worker Context *************************
What does the law say about civil rights and household rights?
 Important laws to achieve the tasks: [Document(page_content="Testo in vigore dal:\n \n\xa020-9-1975\n(CODICE CIVILE-art. 143)\naggiornamenti all'articolo\n Art. 143. \n((Diritti e doveri reciproci dei coniugi.))\n((Con il matrimonio il marito e la moglie acquistano gli stessi diritti e assumono i medesimi doveri. \n Dal matrimonio deriva l'obbligo reciproco alla fedeltà, all'assistenza morale e materiale, alla collaborazione nell'interesse della famiglia e alla coabitazione. \n Entrambi i coniugi sono tenuti, ciascuno in relazione alle proprie sostanze e alla propria capacità di lavoro professionale o casalingo, a contribuire ai bisogni della fam

Text(value='', description='Chat:', placeholder='Type your message')

## Review chat history and Usage Analysis

In [49]:
timer_array

[[datetime.datetime(2024, 7, 3, 8, 54, 54, 997259), 1331, 51, 'assistant'],
 [datetime.datetime(2024, 7, 3, 8, 54, 55, 448118),
  1002,
  483,
  'SIMPLE QUESTION ANSWERING'],
 [datetime.datetime(2024, 7, 3, 8, 55, 24, 840738), 4273, 594, 'assistant'],
 [datetime.datetime(2024, 7, 3, 8, 55, 25, 314685),
  2867,
  706,
  'SIMPLE QUESTION ANSWERING'],
 [datetime.datetime(2024, 7, 3, 8, 56, 11, 186184), 10840, 901, 'assistant'],
 [datetime.datetime(2024, 7, 3, 8, 56, 22, 315474),
  3521,
  1233,
  'REQUIREMENT GATHERING'],
 [datetime.datetime(2024, 7, 3, 9, 30, 49, 840946), 21199, 1513, 'assistant'],
 [datetime.datetime(2024, 7, 3, 9, 30, 49, 865193),
  4175,
  1760,
  'REQUIREMENT GATHERING'],
 [datetime.datetime(2024, 7, 3, 9, 33, 6, 190), 30452, 2100, 'assistant'],
 [datetime.datetime(2024, 7, 3, 9, 33, 6, 511394),
  5464,
  1765,
  'SIMPLE QUESTION ANSWERING']]

In [39]:
assistant.chat_messages

defaultdict(list,
            {<autogen.agentchat.user_proxy_agent.UserProxyAgent at 0x7e9d6371e170>: [{'content': 'hello, how are you.',
               'role': 'user'},
              {'content': 'Hello! How can I assist you today? TERMINATE',
               'role': 'assistant'},
              {'content': 'what does the law say about civil rights and household rights ?',
               'role': 'user'},
              {'function_call': {'arguments': '{"message":"What does the law say about civil rights and household rights?","message_type":"SIMPLE QUESTION ANSWERING","retrival":"TRUE"}',
                'name': 'ask_worker'},
               'content': None,
               'role': 'assistant'},
              {'content': "The Italian Civil Code outlines several important laws regarding civil rights and household rights. Here are the key points:\n\n1. **Diritti e doveri reciproci dei coniugi (Art. 143):**  \n   - Both husband and wife acquire the same rights and responsibilities with marria

In [50]:
print("Actual usage summary for assistant (excluding completion from cache):", assistant.get_actual_usage())
print("Total usage summary for assistant (including completion from cache):", assistant.get_total_usage())

# print("Actual usage summary for worker_user:", worker_user.get_actual_usage())
# print("Total usage summary for worker_user:", worker_user.get_total_usage())

print("Actual usage summary for worker:", worker.get_actual_usage())
print("Total usage summary for worker:", worker.get_total_usage())

# print("Actual usage summary for user_proxy:", user_proxy.get_actual_usage())
# print("Total usage summary for user_proxy:", user_proxy.get_total_usage())
def display_prompt_length(data):
    result = [[0,data[0][1],data[0][2],data[0][3]]]
    #  [data[0]]  # Start with the first entry

    # Track the last seen indices for 'assistant' and 'others'
    last_assistant = data[0]
    last_other = None

    # Iterate over the data list starting from the second item
    for i in range(1, len(data)):
        if data[i][3] == 'assistant':
            if last_assistant:
                # print(last_assistant)
                previous_first_index = last_assistant[1]
                previous_second_index = last_assistant[2]

                current_first_index = data[i][1]
                current_second_index = data[i][2]

                first_index_difference = current_first_index - previous_first_index
                second_index_difference = current_second_index - previous_second_index
                time_taken = (data[i][0] - data[i-1][0]).seconds
                result.append([time_taken, first_index_difference, second_index_difference, data[i][3]])
            last_assistant = data[i]
        else:
            time_taken = (data[i][0] - data[i-1][0]).seconds
            if last_other:
                previous_first_index = last_other[1]
                previous_second_index = last_other[2]

                current_first_index = data[i][1]
                current_second_index = data[i][2]

                first_index_difference = current_first_index - previous_first_index
                second_index_difference = current_second_index - previous_second_index

                result.append([time_taken, first_index_difference, second_index_difference, data[i][3]])
            else:
              result.append([time_taken,data[i][1],data[i][2],data[i][3]])
            last_other = data[i]

    return result

print(display_prompt_length(timer_array))

Actual usage summary for assistant (excluding completion from cache): {'total_cost': 0.21608499999999997, 'gpt-4o-2024-05-13': {'cost': 0.21608499999999997, 'prompt_tokens': 35414, 'completion_tokens': 2601, 'total_tokens': 38015}}
Total usage summary for assistant (including completion from cache): {'total_cost': 0.21608499999999997, 'gpt-4o-2024-05-13': {'cost': 0.21608499999999997, 'prompt_tokens': 35414, 'completion_tokens': 2601, 'total_tokens': 38015}}
Actual usage summary for worker: {'total_cost': 0.011175, 'gpt-4o-2024-05-13': {'cost': 0.011175, 'prompt_tokens': 654, 'completion_tokens': 527, 'total_tokens': 1181}}
Total usage summary for worker: {'total_cost': 0.053794999999999996, 'gpt-4o-2024-05-13': {'cost': 0.053794999999999996, 'prompt_tokens': 5464, 'completion_tokens': 1765, 'total_tokens': 7229}}
[[0, 1331, 51, 'assistant'], [0, 1002, 483, 'SIMPLE QUESTION ANSWERING'], [29, 2942, 543, 'assistant'], [0, 1865, 223, 'SIMPLE QUESTION ANSWERING'], [45, 6567, 307, 'assistan

In [51]:
from autogen import AssistantAgent, OpenAIWrapper, UserProxyAgent, gather_usage_summary
usage_summary = gather_usage_summary([assistant, worker])
usage_summary["usage_including_cached_inference"]

{'total_cost': 0.26987999999999995,
 'gpt-4o-2024-05-13': {'cost': 0.26987999999999995,
  'prompt_tokens': 40878,
  'completion_tokens': 4366,
  'total_tokens': 45244}}

## Creating a second Assistant Agent to demonstrate chat continuance

In [52]:
# create an AssistantAgent instance named "assistant"
#  Your tasks:
#     1. Identify the steps needed to solve the user's request.
#     2. Assign each step to the ask_worker function, which will return the results upon task completion.
#     3. Always use 'VERIFICATION' worker to verify each result from ask_worker function by giving it the task set out to do with what has been done to verify the task has been done correctly.
#     4. Always use 'COMMUNICATION' worker to get you what to say to the user and enable you to directly communicate with user.


# If you need to ask the user a question or you have finished the task, ensure everything is in English plus make sure you used 'COMMUNICATION' workers response only and add TERMINATE at the end.
test_assistant_2 = autogen.AssistantAgent(
   name="assistant",
    code_execution_config=False,
    system_message=
    """
    You are a Legal Assistant with advanced capabilities to solve complex problems. Your primary objective is to plan steps to appropriately solve users'\
     requests or tasks using worker function calling.

    The main problems you will be solving include:
    - Answering law questions

    Available Workers:
        - REQUIREMENT GATHERING - will enable you to ask relevant questions to the user based on the task
        - SIMPLE QUESTION ANSWERING - will enable you to answer simple questions and law retrievals to the user based on the task

    Your tasks:
    1. Outline the steps required to address the user's request.
    2. Use the `ask_worker` function to delegate each step, ensuring it returns the results upon task completion.
    3. VERY IMPORTANT: Before you return any result ensure that you do not need worker skills to perform a task.

    Key guidelines:
    - ALWAYS use the ask_worker function for different tasks and send it only the most important instructions and commands.
    - FORCE the ask_worker function to only give you the answer for your request without adding explanation or guidance what it achieved.
    - Give the worker the advantage of article retrival where it will get the latest laws regarding the task and be able to accomplish the task.
    - If the database retrieval does not provide important laws for the task, inform the user that you were not able to find the relevant law at that moment.
    - If results are based on an article or law codes, mention the articles in your response and the worker's response so that the user can verify the information.
    - DO NOT forget to put TERMINATE at the end, If you want feedback from user

    Communication:
    - Your answer should be constructed in the given language the user used to ask.
    - Before responding to the user make sure you used all the skills the worker function provides.
    - When communicating with the Worker make sure to send The most Important context to it.

    If you need to ask the user a question or you have finished the task add TERMINATE at the end.
    MUST - If the user send you an empty message just return your previous message and add TERMINATE at the end.
    """,

    llm_config={
        "temperature": 0,
        "timeout": 600,
        "cache_seed": None,
        "config_list": config_list,
        "functions": [
             {
                        "name": "ask_worker",
                        "description": "ask worker to:  1. generate questions for requirement gathering, 2. retrieve relevant laws based on the topic sent as message",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "message": {
                                    "type": "string",
                                    "description": "question to ask worker. Make sure the question include enough context, such as the required task. The worker does not know the conversation between you and the user, unless you share the conversation with the worker. Ensure the original question of the user is also passed as context such that the worker is generating intermediate steps necessary to address it. It is VERY IMPORTANT that you provide a clear and autoritative feedback to the worker to stay focused on answering the original question. If the worker is wandering off from the mission, terminate!"
                                },
                                "message_type": {
                                    "type": "string",
                                    "description": "Type of work you want it to achieve. Options are: 'REQUIREMENT GATHERING','SIMPLE QUESTION ANSWERING'"
                                },
                                "retrival":{
                                    "type": "string",
                                    "description": "Becomes 'TRUE' or 'FALSE' which would indicate to perform retrieval of relevant articles from the law to achieve the task"
                                },
                            },
                            "required": ["message","message_type","retrival"]
                        }
                        }
        ],
    },

)

# def termination_msg(x):
#     return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()


# create a UserProxyAgent instance named "user_proxy"
test_user_proxy_2 = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=termination_msg,
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=3,
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    # code_execution_config={"work_dir": "planning"},
    function_map={"ask_worker": ask_worker },
    # chat_messages=None
)
full_user_proxy_history = [messages for agent, messages in user_proxy.chat_messages.items()][0]
full_assistance_history = [messages for agent, messages in assistant.chat_messages.items()][0]
print(test_assistant_2.chat_messages)
test_user_proxy_2.chat_messages

defaultdict(<class 'list'>, {})


defaultdict(list, {})

In [53]:
from collections import defaultdict

user_proxy_history = defaultdict(list, {
    test_user_proxy_2: full_user_proxy_history
})

assistant_history = defaultdict(list, {
    test_assistant_2: full_assistance_history
})

test_assistant_2._oai_messages = user_proxy_history
test_user_proxy_2._oai_messages = assistant_history
len(test_user_proxy_2.chat_messages)

1

In [55]:
test_user_proxy_2.chat_messages

defaultdict(list,
            {<autogen.agentchat.assistant_agent.AssistantAgent at 0x7e9d6384ab60>: [{'content': 'hello, how are you.',
               'role': 'user'},
              {'content': 'Hello! How can I assist you today? TERMINATE',
               'role': 'assistant'},
              {'content': 'what does the law say about civil rights and household rights ?',
               'role': 'user'},
              {'function_call': {'arguments': '{"message":"What does the law say about civil rights and household rights?","message_type":"SIMPLE QUESTION ANSWERING","retrival":"TRUE"}',
                'name': 'ask_worker'},
               'content': None,
               'role': 'assistant'},
              {'content': "The Italian Civil Code outlines several important laws regarding civil rights and household rights. Here are the key points:\n\n1. **Diritti e doveri reciproci dei coniugi (Art. 143):**  \n   - Both husband and wife acquire the same rights and responsibilities with marriag

In [56]:
test_user_proxy_2.initiate_chat(test_assistant_2, clear_history=False,message="So Does Men and Women have equal rights in a household?")

user_proxy (to assistant):

So Does Men and Women have equal rights in a household?

--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function call: ask_worker *****
Arguments: 
{"message":"Do men and women have equal rights in a household according to the law?","message_type":"SIMPLE QUESTION ANSWERING","retrival":"TRUE"}
***********************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION ask_worker...
************** SIMPLE QUESTION ANSWERING *************************
Do men and women have equal rights in a household according to the law?
TRUE
retrival started
retrival ended
 ************** Fully Prepared Worker Context *************************
Do men and women have equal rights in a household according to the law?
 Important laws to achieve the tasks: [Document(page_content="Testo in vigor

ChatResult(chat_id=None, chat_history=[{'content': 'hello, how are you.', 'role': 'user'}, {'content': 'Hello! How can I assist you today? TERMINATE', 'role': 'assistant'}, {'content': 'what does the law say about civil rights and household rights ?', 'role': 'user'}, {'function_call': {'arguments': '{"message":"What does the law say about civil rights and household rights?","message_type":"SIMPLE QUESTION ANSWERING","retrival":"TRUE"}', 'name': 'ask_worker'}, 'content': None, 'role': 'assistant'}, {'content': "The Italian Civil Code outlines several important laws regarding civil rights and household rights. Here are the key points:\n\n1. **Diritti e doveri reciproci dei coniugi (Art. 143):**  \n   - Both husband and wife acquire the same rights and responsibilities with marriage.\n   - Marriage entails mutual obligations for fidelity, moral and material support, collaboration for the family's interest, and cohabitation.\n   - Each spouse must contribute to the family's needs accordin

In [57]:
test_user_proxy_2._oai_messages

defaultdict(list,
            {<autogen.agentchat.assistant_agent.AssistantAgent at 0x7e9d6384ab60>: [{'content': 'hello, how are you.',
               'role': 'user'},
              {'content': 'Hello! How can I assist you today? TERMINATE',
               'role': 'assistant'},
              {'content': 'what does the law say about civil rights and household rights ?',
               'role': 'user'},
              {'function_call': {'arguments': '{"message":"What does the law say about civil rights and household rights?","message_type":"SIMPLE QUESTION ANSWERING","retrival":"TRUE"}',
                'name': 'ask_worker'},
               'content': None,
               'role': 'assistant'},
              {'content': "The Italian Civil Code outlines several important laws regarding civil rights and household rights. Here are the key points:\n\n1. **Diritti e doveri reciproci dei coniugi (Art. 143):**  \n   - Both husband and wife acquire the same rights and responsibilities with marriag

## Future Works and Plans

- Using other LLMs (other than OpenAI's)
- Multimedia support espescially Voice (looking at GPT-4o)
